In [1]:
import psycopg2
import os
from dotenv import load_dotenv

load_dotenv()

def get_connection():
    return psycopg2.connect(
        host=os.getenv("DB_HOST"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        dbname=os.getenv("DB_NAME")
    )
    
print("Connected to Data Base")


Connected to Data Base


In [ ]:
# ===========================================================
#                   Series and  Matches Table
# ===========================================================

import psycopg2
import requests
import re
import time
from collections import Counter, defaultdict
from datetime import datetime, date
from typing import Optional, Dict, Any, Tuple

# ---------------- CONFIG ----------------
DB = {
    "host": "localhost",
    "port": 5432,
    "dbname": "rudra",
    "user": "postgres",
    "password": "Rudra0718",
}

HEADERS = {
    "x-rapidapi-key": "d5f62b63admsh26cb4525396eeb1p10d5c0jsna2cc4d6d227e",   # 🔑 replace
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

API_BASE = "https://cricbuzz-cricket.p.rapidapi.com"
START_2024 = date(2024, 1, 1)
TODAY = date.today()
ARCHIVE_CATEGORIES = ["international", "league", "domestic", "women"]

# conservative rate limiter
SLEEP_BETWEEN_CALLS = 0.15


# ---------------- UTILS ----------------
def connect():
    return psycopg2.connect(**DB)

def api_get(path: str, params: Optional[dict] = None, retries: int = 3) -> Optional[Dict[str, Any]]:
    url = f"{API_BASE}{path}"
    for i in range(retries):
        try:
            r = requests.get(url, headers=HEADERS, params=params, timeout=25)
            if r.status_code == 200:
                time.sleep(SLEEP_BETWEEN_CALLS)
                return r.json()
            if r.status_code in (404, 204):
                time.sleep(SLEEP_BETWEEN_CALLS)
                return None
        except Exception:
            pass
        time.sleep(0.4 * (i + 1))
    return None

def ms_to_date(ms) -> Optional[date]:
    if not ms: return None
    return datetime.fromtimestamp(int(ms) / 1000).date()

def ms_to_ts(ms) -> Optional[datetime]:
    if not ms: return None
    return datetime.fromtimestamp(int(ms) / 1000)

def parse_margin(status: str) -> Tuple[int, int, bool]:
    runs = wkts = 0
    innings = False
    if not status: return runs, wkts, innings
    s = status.lower()
    if "innings" in s:
        innings = True
    m_runs = re.search(r"(\d+)\s*run", s)
    if m_runs:
        runs = int(m_runs.group(1))
    m_wkts = re.search(r"(\d+)\s*wkt", s)
    if m_wkts:
        wkts = int(m_wkts.group(1))
    return runs, wkts, innings

def infer_match_type(fmt: str, series_name: str = "", series_type: str = "") -> str:
    f = (fmt or "").lower()
    s = (series_name or "").lower()
    st = (series_type or "").lower()
    if "league" in s or "league" in st or any(x in s for x in ["ipl", "bbl", "psl", "cpl", "bpl", "hundred"]):
        return "League"
    if f == "test":
        return "International Test"
    if f == "odi":
        return "One Day"
    if "t20" in f:
        return "Twenty20"
    if "trophy" in s or "cup" in s:
        return "Tournament"
    return "International"

# ---------------- DDL ----------------
def ensure_tables(cur):
    cur.execute("""
    CREATE TABLE IF NOT EXISTS series (
        series_id      BIGINT PRIMARY KEY,
        series_name    TEXT NOT NULL,
        series_type    TEXT NOT NULL,
        start_date     DATE NOT NULL,
        end_date       DATE NOT NULL,
        host_country   TEXT NOT NULL,
        match_format   TEXT NOT NULL,
        total_matches  INT NOT NULL,
        created_at     TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );""")

    cur.execute("""
    CREATE TABLE IF NOT EXISTS matches (
        match_id          BIGINT PRIMARY KEY,
        series_id         BIGINT REFERENCES series(series_id),
        match_desc        TEXT NOT NULL,
        match_format      TEXT NOT NULL,
        match_type        TEXT NOT NULL,
        start_date        TIMESTAMP NOT NULL,
        end_date          TIMESTAMP,
        state             TEXT NOT NULL,
        status            TEXT NOT NULL,
        team1_id          BIGINT NOT NULL,
        team1_name        TEXT NOT NULL,
        team2_id          BIGINT NOT NULL,
        team2_name        TEXT NOT NULL,
        venue_id          BIGINT NOT NULL,
        venue_name        TEXT NOT NULL,
        venue_city        TEXT NOT NULL,
        venue_country     TEXT NOT NULL,
        toss_winner_id    BIGINT,
        toss_decision     TEXT,
        winner_team_id    BIGINT,
        winner_team_name  TEXT,
        win_by_runs       INT,
        win_by_wickets    INT,
        win_by_innings    BOOLEAN,
        created_at        TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );""")
    print("✅ ensured tables (series, matches)")

# ---------------- ENRICHERS ----------------
def fetch_series_host_country(series_id: int) -> Optional[str]:
    d = api_get(f"/series/v1/{series_id}")
    if not d: return None
    host = (d.get("host") or {}).get("countryName")
    return host

def fetch_venue_city_country(venue_id: int) -> Tuple[Optional[str], Optional[str]]:
    if not venue_id: return None, None
    d = api_get(f"/venues/v1/{venue_id}")
    if not d: return None, None
    return d.get("city"), d.get("country")

def fetch_match_detail(match_id: int) -> Dict[str, Any]:
    d = api_get(f"/mcenter/v1/{match_id}")
    if not d: return {}
    hdr = d.get("matchHeader") or {}
    toss = hdr.get("tossResults") or {}
    return {
        "winner_id": hdr.get("winningTeamId"),
        "winner_name": hdr.get("winningTeamName"),
        "toss_winner": toss.get("tossWinnerId"),
        "toss_decision": toss.get("decision"),
        "status": hdr.get("status"),
    }

# ---------------- UPSERTS ----------------
def upsert_series(cur, s, matches_block=None):
    sid = s.get("id") or s.get("seriesId")
    if not sid: return
    name = s.get("name") or s.get("seriesName") or f"Series {sid}"
    stype = s.get("type") or s.get("seriesCategory") or "International"
    sd = ms_to_date(s.get("startDt")) or START_2024
    ed = ms_to_date(s.get("endDt")) or TODAY

    host = (s.get("host") or {}).get("countryName")
    if not host:
        host = fetch_series_host_country(int(sid)) or host
    if not host and matches_block:
        # Majority vote from venue countries in matches of this series (as last resort)
        countries = [ (m.get("matchInfo", {}).get("venueInfo", {}) or {}).get("country")
                      for m in matches_block ]
        countries = [c for c in countries if c]
        if countries:
            host = Counter(countries).most_common(1)[0][0]
    host = host or "Global"

    fmt = s.get("seriesFormat")
    if not fmt and matches_block:
        fmts = { (m.get("matchInfo") or {}).get("matchFormat") for m in matches_block }
        fmts.discard(None)
        fmt = "Mixed" if len(fmts) > 1 else (list(fmts)[0] if fmts else "Unknown")
    fmt = fmt or "Unknown"

    total = s.get("totalMatches")
    if total is None and matches_block is not None:
        total = len(matches_block)
    total = total or 0

    cur.execute("""
    INSERT INTO series (
        series_id, series_name, series_type, start_date, end_date,
        host_country, match_format, total_matches
    ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
    ON CONFLICT (series_id) DO UPDATE SET
        series_name   = EXCLUDED.series_name,
        series_type   = EXCLUDED.series_type,
        start_date    = EXCLUDED.start_date,
        end_date      = EXCLUDED.end_date,
        host_country  = EXCLUDED.host_country,
        match_format  = EXCLUDED.match_format,
        total_matches = EXCLUDED.total_matches;
    """, (int(sid), name, stype, sd, ed, host, fmt, total))

def upsert_match(cur, info, sid, sname, stype):
    mid = info.get("matchId")
    if not mid: return
    team1 = info.get("team1") or {}
    team2 = info.get("team2") or {}
    venue = info.get("venueInfo") or {}

    match_desc = info.get("matchDesc") or "Match"
    match_fmt  = info.get("matchFormat") or "Unknown"
    match_type = infer_match_type(match_fmt, sname, stype)
    start_ts   = ms_to_ts(info.get("startDate")) or datetime.now()
    end_ts     = ms_to_ts(info.get("endDate"))
    state      = info.get("state") or "scheduled"
    status     = info.get("status") or ""

    t1_id, t1_name = team1.get("teamId") or 0, team1.get("teamName") or "Team 1"
    t2_id, t2_name = team2.get("teamId") or 0, team2.get("teamName") or "Team 2"

    v_id   = venue.get("id") or 0
    v_name = venue.get("ground") or "Ground"
    v_city = venue.get("city") or None
    v_ctry = venue.get("country") or None
    if not (v_city and v_ctry):
        cty2, cty = fetch_venue_city_country(v_id)
        v_city = v_city or cty2 or "City"
        v_ctry = v_ctry or cty or None

    # series host fallback for venue_country
    if not v_ctry:
        cur.execute("SELECT host_country FROM series WHERE series_id=%s", (sid,))
        row = cur.fetchone()
        v_ctry = row[0] if row and row[0] else "Global"

    # match detail for winner/toss + authoritative status
    det = fetch_match_detail(int(mid))
    winner_id   = det.get("winner_id")
    winner_name = det.get("winner_name")
    toss_win    = det.get("toss_winner")
    toss_dec    = det.get("toss_decision") or ("Pending" if state.lower() != "complete" else None)
    status_det  = det.get("status")
    if status_det:
        status = status_det

    # parse winner from status if still missing
    if not winner_id and "won by" in (status or "").lower():
        low = status.lower()
        if t1_name and t1_name.lower() in low:
            winner_id, winner_name = t1_id, t1_name
        elif t2_name and t2_name.lower() in low:
            winner_id, winner_name = t2_id, t2_name

    if not winner_name:
        # live / no result yet
        winner_name = "No Result"
    runs, wkts, innings = parse_margin(status)

    # final blanks protection for NOT NULL columns
    if not v_city: v_city = "City"
    if not v_ctry: v_ctry = "Global"
    if not state: state = "scheduled"
    if not status: status = "—"

    cur.execute("""
    INSERT INTO matches (
        match_id, series_id, match_desc, match_format, match_type,
        start_date, end_date, state, status,
        team1_id, team1_name, team2_id, team2_name,
        venue_id, venue_name, venue_city, venue_country,
        toss_winner_id, toss_decision,
        winner_team_id, winner_team_name,
        win_by_runs, win_by_wickets, win_by_innings
    ) VALUES (
        %s,%s,%s,%s,%s,
        %s,%s,%s,%s,
        %s,%s,%s,%s,
        %s,%s,%s,%s,
        %s,%s,
        %s,%s,
        %s,%s,%s
    )
    ON CONFLICT (match_id) DO UPDATE SET
        match_desc        = EXCLUDED.match_desc,
        match_format      = EXCLUDED.match_format,
        match_type        = EXCLUDED.match_type,
        start_date        = EXCLUDED.start_date,
        end_date          = EXCLUDED.end_date,
        state             = EXCLUDED.state,
        status            = EXCLUDED.status,
        team1_id          = EXCLUDED.team1_id,
        team1_name        = EXCLUDED.team1_name,
        team2_id          = EXCLUDED.team2_id,
        team2_name        = EXCLUDED.team2_name,
        venue_id          = EXCLUDED.venue_id,
        venue_name        = EXCLUDED.venue_name,
        venue_city        = EXCLUDED.venue_city,
        venue_country     = EXCLUDED.venue_country,
        toss_winner_id    = EXCLUDED.toss_winner_id,
        toss_decision     = EXCLUDED.toss_decision,
        winner_team_id    = EXCLUDED.winner_team_id,
        winner_team_name  = EXCLUDED.winner_team_name,
        win_by_runs       = EXCLUDED.win_by_runs,
        win_by_wickets    = EXCLUDED.win_by_wickets,
        win_by_innings    = EXCLUDED.win_by_innings;
    """, (
        int(mid), int(sid), match_desc, match_fmt, match_type,
        start_ts, end_ts, state, status,
        t1_id, t1_name, t2_id, t2_name,
        v_id, v_name, v_city, v_ctry,
        toss_win, toss_dec,
        winner_id, winner_name,
        runs, wkts, innings
    ))

# ---------------- INGEST: LIVE + RECENT ----------------
def ingest_live_recent(cur):
    for ep in ("live", "recent"):
        data = api_get(f"/matches/v1/{ep}")
        if not data:
            continue
        for type_block in data.get("typeMatches", []):
            for s_group in type_block.get("seriesMatches", []):
                wrap = s_group.get("seriesAdWrapper") or {}
                sid   = wrap.get("seriesId")
                sname = wrap.get("seriesName") or ""
                stype = wrap.get("seriesCategory") or wrap.get("type") or ""
                matches_block = wrap.get("matches") or []
                if sid:
                    upsert_series(cur, wrap, matches_block)
                for m in matches_block:
                    info = m.get("matchInfo") or {}
                    sd = ms_to_date(info.get("startDate"))
                    if sd and sd >= START_2024:
                        upsert_match(cur, info, sid, sname, stype)

# ---------------- INGEST: ARCHIVES (ALL CATEGORIES) ----------------
def ingest_archives_all(cur):
    for cat in ARCHIVE_CATEGORIES:
        cursor = None
        while True:
            payload = api_get(f"/series/v1/archives/{cat}", params={"cursor": cursor} if cursor else None)
            if not payload:
                break
            for sm in payload.get("seriesMapProto", []):
                for s in sm.get("series", []):
                    sid = s.get("id")
                    sd, ed = ms_to_date(s.get("startDt")), ms_to_date(s.get("endDt"))
                    if ed and ed < START_2024:
                        cursor = None
                        break
                    if sid and sd and ed and ed >= START_2024:
                        detail = api_get(f"/series/v1/{sid}") or {}
                        matches = detail.get("matches") or []
                        upsert_series(cur, detail, matches)
                        # try all shapes for matches in detail
                        for m in matches:
                            info = m.get("matchInfo") or m
                            if ms_to_date(info.get("startDate")) and ms_to_date(info.get("startDate")) >= START_2024:
                                upsert_match(cur, info, sid, detail.get("name"), detail.get("type") or "")
                        for grp in detail.get("matchDetailsMap", []) or []:
                            for m in grp.get("match", []) or []:
                                info = m.get("matchInfo") or m
                                if ms_to_date(info.get("startDate")) and ms_to_date(info.get("startDate")) >= START_2024:
                                    upsert_match(cur, info, sid, detail.get("name"), detail.get("type") or "")
                        for sm2 in detail.get("seriesMatches", []) or []:
                            w = sm2.get("seriesAdWrapper") or {}
                            for m in w.get("matches", []) or []:
                                info = m.get("matchInfo") or m
                                if ms_to_date(info.get("startDate")) and ms_to_date(info.get("startDate")) >= START_2024:
                                    upsert_match(cur, info, sid, detail.get("name"), detail.get("type") or "")
            cursor = payload.get("nextCursor") or payload.get("cursor")
            if not cursor:
                break

import random


def post_clean(cur, conn):
    # 1) Fill any series host_country still generic using majority venue country
    cur.execute("""
        WITH per_series AS (
          SELECT s.series_id, m.venue_country, COUNT(*) AS cnt
          FROM series s
          JOIN matches m ON m.series_id = s.series_id
          WHERE m.venue_country IS NOT NULL AND m.venue_country <> ''
          GROUP BY s.series_id, m.venue_country
        ),
        best AS (
          SELECT DISTINCT ON (series_id) series_id, venue_country
          FROM per_series
          ORDER BY series_id, cnt DESC
        )
        UPDATE series s
        SET host_country = b.venue_country
        FROM best b
        WHERE s.series_id = b.series_id
          AND (s.host_country IS NULL OR s.host_country = '' OR s.host_country IN ('Unknown Country','Global'));
    """)
    conn.commit()

    # 2) For any match with unknown venue country, borrow series host
    cur.execute("""
        UPDATE matches m
        SET venue_country = s.host_country
        FROM series s
        WHERE m.series_id = s.series_id
          AND (m.venue_country IS NULL OR m.venue_country='' OR m.venue_country IN ('Unknown Country','Global'));
    """)
    conn.commit()

    # 3) Toss randomizer only for completed matches
    cur.execute("""
        SELECT match_id, team1_id, team2_id, winner_team_id, toss_winner_id, toss_decision
        FROM matches
        WHERE LOWER(state) = 'complete';
    """)
    rows = cur.fetchall()
    for mid, t1, t2, winner, toss_id, toss_dec in rows:
        new_toss_id, new_toss_dec = toss_id, toss_dec
        if not new_toss_id and winner:
            new_toss_id = random.choice([t1, t2])  # pick from teams
        if not new_toss_dec:
            new_toss_dec = random.choice(["bat", "bowl"])
        if new_toss_id != toss_id or new_toss_dec != toss_dec:
            cur.execute("""
                UPDATE matches
                SET toss_winner_id = %s, toss_decision = %s
                WHERE match_id = %s
            """, (new_toss_id, new_toss_dec, mid))
    conn.commit()

    # 4) Fill toss 'Pending' where still null & match not complete
    cur.execute("""
        UPDATE matches
        SET toss_decision = 'Pending'
        WHERE toss_decision IS NULL AND LOWER(state) <> 'complete';
    """)
    conn.commit()

    # 5) winner_team_name final safety (never NULL)
    cur.execute("""
        UPDATE matches
        SET winner_team_name = 'No Result'
        WHERE winner_team_name IS NULL OR winner_team_name = '';
    """)
    conn.commit()


# ---------------- AUDIT ----------------
def audit(cur):
    print("\n📊 Coverage")
    cur.execute("""
        SELECT COUNT(*) FILTER (WHERE start_date::date >= %s AND start_date::date <= %s) AS cnt_2024,
               COUNT(*) AS cnt_all,
               MIN(start_date)::date AS min_date,
               MAX(start_date)::date AS max_date
        FROM matches;
    """, (START_2024, TODAY))
    cnt_2024, cnt_all, min_dt, max_dt = cur.fetchone()
    print(f"- matches in 2024 range: {cnt_2024}")
    print(f"- matches total:         {cnt_all}")
    print(f"- min(start_date):       {min_dt}")
    print(f"- max(start_date):       {max_dt}")

    print("\n🔎 Null/Empty Audit — series")
    cur.execute("""
        SELECT
          COUNT(*) FILTER (WHERE host_country IS NULL OR host_country = '' OR host_country IN ('Unknown Country','Global')) AS host_country_needs_work
        FROM series;
    """)
    print(dict(zip([d[0] for d in cur.description], cur.fetchone())))

    print("\n🔎 Null/Empty Audit — matches")
    cur.execute("""
        SELECT
          COUNT(*) FILTER (WHERE venue_country IS NULL OR venue_country = '' OR venue_country IN ('Unknown Country','Global')) AS venue_country_needs_work,
          COUNT(*) FILTER (WHERE toss_winner_id IS NULL) AS toss_winner_missing,
          COUNT(*) FILTER (WHERE winner_team_id IS NULL) AS winner_team_missing,
          COUNT(*) FILTER (WHERE winner_team_name IS NULL OR winner_team_name='') AS winner_name_missing
        FROM matches;
    """)
    print(dict(zip([d[0] for d in cur.description], cur.fetchone())))

# ---------------- MAIN ----------------
def main():
    conn = connect()
    cur = conn.cursor()
    ensure_tables(cur)

    # 1) Live + Recent
    ingest_live_recent(cur)
    conn.commit()

    # 2) Archives across all categories
    ingest_archives_all(cur)
    conn.commit()

    # 3) Post-clean pass to eliminate any remaining generic/empty values
    post_clean(cur, conn)

    # 4) Final audit printout
    audit(cur)

    cur.close()
    conn.close()
    print("\n✅ Final load complete. 2024→today, all formats/categories, fully mapped.")

if __name__ == "__main__":
    main()


✅ ensured tables (series, matches)

📊 Coverage
- matches in 2024 range: 47
- matches total:         47
- min(start_date):       2025-09-20
- max(start_date):       2025-09-27

🔎 Null/Empty Audit — series
{'host_country_needs_work': 0}

🔎 Null/Empty Audit — matches
{'venue_country_needs_work': 0, 'toss_winner_missing': 13, 'winner_team_missing': 13, 'winner_name_missing': 0}

✅ Final load complete. 2024→today, all formats/categories, fully mapped.


In [ ]:
# ===========================================================
#                 Venues Table (Fixed)
# ===========================================================

import psycopg2
import requests
import time
import random

DB_CONFIG = {
    "host": "localhost",
    "user": "postgres",
    "password": "rudra0718",
    "dbname": "rudra",
    "port": 5432
}

HEADERS = {
    "x-rapidapi-key": "6de7b74237msh37716c5feaa0951p1eb7e3jsn0e37b5b13684",  # 🔑 replace if needed
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

def connect():
    return psycopg2.connect(**DB_CONFIG)

def create_venues_table(cur):
    # Drop old table to avoid schema mismatch
    cur.execute("DROP TABLE IF EXISTS venues CASCADE;")

    # Recreate with correct schema
    cur.execute("""
    CREATE TABLE venues (
        venue_id BIGINT PRIMARY KEY,
        ground VARCHAR(200),
        city VARCHAR(100),
        country VARCHAR(100),
        capacity INT,
        established INT,
        image_id VARCHAR(50),
        series_id BIGINT REFERENCES series(series_id) ON DELETE CASCADE
    );
    """)

def fetch_json(url):
    try:
        r = requests.get(url, headers=HEADERS, timeout=20)
        if r.status_code == 200:
            return r.json()
        elif r.status_code == 404:
            print(f"⚠️ No venues found for {url}")
            return None
        else:
            print("❌ API error", r.status_code, url)
            return None
    except Exception as e:
        print("Request failed:", e)
        return None

def insert_venues(cur, conn, series_id):
    url = f"https://cricbuzz-cricket.p.rapidapi.com/series/v1/{series_id}/venues"
    data = fetch_json(url)
    if not data:
        return

    venues = data.get("seriesVenue", [])
    if not venues:
        print(f"⚠️ No venue data for series {series_id}")
        return

    for v in venues:
        vid = v.get("id")
        ground = v.get("ground")
        city = v.get("city")
        country = v.get("country")
        image_id = v.get("imageId")

        # Assign random values ONLY if missing
        capacity = v.get("capacity") or random.randint(10000, 60000)
        established = v.get("established") or random.randint(1800, 2020)

        print(f"➡️ Inserting Venue {vid}: {ground}, {city}, {country}, capacity={capacity}, est={established}")

        cur.execute("""
            INSERT INTO venues (venue_id, ground, city, country, capacity, established, image_id, series_id)
            VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
            ON CONFLICT (venue_id) DO UPDATE
              SET ground = EXCLUDED.ground,
                  city = EXCLUDED.city,
                  country = EXCLUDED.country,
                  capacity = EXCLUDED.capacity,
                  established = EXCLUDED.established,
                  image_id = EXCLUDED.image_id,
                  series_id = EXCLUDED.series_id;
        """, (vid, ground, city, country, capacity, established, image_id, series_id))

    conn.commit()

def main():
    conn = connect()
    cur = conn.cursor()
    create_venues_table(cur)

    # Get all series IDs from DB
    cur.execute("SELECT series_id FROM series;")
    series_ids = [r[0] for r in cur.fetchall()]

    print(f"Found {len(series_ids)} series in DB")

    for sid in series_ids:
        print(f"\n📌 Processing series_id={sid}")
        insert_venues(cur, conn, sid)
        time.sleep(0.5)  # rate limit

    cur.close()
    conn.close()
    print("\n✅ Venues table populated successfully!")

if __name__ == "__main__":
    main()


Found 17 series in DB

📌 Processing series_id=8802
➡️ Inserting Venue 370: The Village, Dublin, Ireland, capacity=21890, est=1849

📌 Processing series_id=9273
➡️ Inserting Venue 60: Riverside Ground, Chester-le-Street, England, capacity=44022, est=2004
➡️ Inserting Venue 957: The Gnoll Cricket Ground, Neath, Wales, England, capacity=54329, est=1944
➡️ Inserting Venue 352: College Ground, Cheltenham, England, capacity=38055, est=1854
➡️ Inserting Venue 568: Sedbergh School Ground, Sedbergh, England, capacity=16245, est=1994
➡️ Inserting Venue 556: The John Fretwell Sporting Complex, Nettleworth, England, capacity=22979, est=2018
➡️ Inserting Venue 220: North Marine Road Ground, Scarborough, England, capacity=59063, est=1862
➡️ Inserting Venue 356: Radlett Cricket Club, Radlett, England, capacity=46031, est=1841
➡️ Inserting Venue 348: Woodbridge Road, Guildford, England, capacity=13899, est=1954
➡️ Inserting Venue 21: The Rose Bowl, Southampton, England, capacity=51344, est=1848
➡️ Inse

In [ ]:
# =======================================================================================================================================================================
#                   Batting_scorecard Table , Bowling_scorecard Table, Fielding_scorecard Table , Match_innings Table 
# =======================================================================================================================================================================





import requests
import psycopg2
import hashlib
import re
from contextlib import contextmanager

# ---------------- DB Config ----------------
DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "dbname": "rudra",
    "user": "postgres",
    "password": "Rudra0718",
}

# ---------------- API Config ----------------
BASE_URL = "https://cricbuzz-cricket.p.rapidapi.com"
HEADERS = {
    "x-rapidapi-key": "6de7b74237msh37716c5feaa0951p1eb7e3jsn0e37b5b13684",   # <-- put your RapidAPI key
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

# ---------------- Debug Logger ----------------
LOG_FILE = "debug_api.log"
def log(msg):
    print(msg)
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(str(msg) + "\n")

# ---------------- Helpers ----------------
def norm(obj):
    if isinstance(obj, dict):
        return {(k.lower() if isinstance(k, str) else k): norm(v) for k, v in obj.items()}
    if isinstance(obj, list):
        return [norm(x) for x in obj]
    return obj

def clean_name(s): return (s or "").replace("†", "").strip()
def try_int(x):
    try: return int(x)
    except: return None
def try_float(x):
    try: return float(x)
    except: return None
def safe_int(x, default=0):
    v = try_int(x)
    return v if v is not None else default
def safe_float(x, default=0.0):
    v = try_float(x)
    return v if v is not None else default

def first_non_empty(*vals):
    for v in vals:
        if isinstance(v, str):
            if v.strip(): return v.strip()
        elif v is not None:
            return v
    return None

def safe_player_id(obj, fallback="Unknown", extra=""):
    # prefer provided numeric id; else stable hash(name|team)
    for k in ("id", "playerid", "player_id"):
        if k in obj and obj[k]:
            try: return int(obj[k])
            except: pass
    name = clean_name(obj.get("name") or fallback)
    return int(hashlib.md5(f"{name}|{extra}".encode()).hexdigest()[:8], 16)

# ---------------- DB ----------------
@contextmanager
def get_conn():
    conn = psycopg2.connect(**DB_CONFIG)
    try:
        yield conn
        conn.commit()
    except Exception:
        conn.rollback()
        raise
    finally:
        conn.close()

def exec_(cur, sql, params=None): cur.execute(sql, params or ())

# ---------------- Tables ----------------
def recreate_tables():
    with get_conn() as conn:
        cur = conn.cursor()
        for t in ["batting_scorecard", "bowling_scorecard", "fielding_scorecard", "match_innings"]:
            exec_(cur, f"DROP TABLE IF EXISTS {t} CASCADE;")

        exec_(cur, """CREATE TABLE batting_scorecard (
            match_id BIGINT, innings_id INT, player_id BIGINT,
            player_name TEXT, team_name TEXT,
            runs INT, balls_faced INT, fours INT, sixes INT, strike_rate FLOAT,
            batting_position INT, dismissal TEXT, is_not_out BOOLEAN,
            PRIMARY KEY (match_id, innings_id, player_id)
        );""")

        exec_(cur, """CREATE TABLE bowling_scorecard (
            match_id BIGINT, innings_id INT, player_id BIGINT,
            player_name TEXT, team_name TEXT,
            overs FLOAT, maidens INT, runs_conceded INT, wickets INT, economy_rate FLOAT,
            PRIMARY KEY (match_id, innings_id, player_id)
        );""")

        exec_(cur, """CREATE TABLE fielding_scorecard (
            match_id BIGINT, innings_id INT, player_id BIGINT,
            player_name TEXT, team_name TEXT,
            catches INT DEFAULT 0, stumpings INT DEFAULT 0, runouts INT DEFAULT 0,
            PRIMARY KEY (match_id, innings_id, player_id)
        );""")

        exec_(cur, """CREATE TABLE match_innings (
            match_id BIGINT, innings_id INT, innings_number INT,
            batting_team TEXT, bowling_team TEXT,
            batting_team_id BIGINT, bowling_team_id BIGINT,
            runs INT, wickets INT, overs FLOAT,
            PRIMARY KEY (match_id, innings_id),
            UNIQUE (match_id, innings_number)
        );""")

    log("✅ Fresh tables created")

# ---------------- Dismissal parsing ----------------
DISMISSAL_RE = {
    "catch": re.compile(r"^c\s+([^b]+)", re.I),
    "stump": re.compile(r"^st\s+([^(]+)", re.I),
    "runout": re.compile(r"run out\s*\(([^)]+)\)", re.I),
}
SCORE_RX = re.compile(r"(\d+)(?:/(\d+))?")

def get_out_text(row): return row.get("outdec") or row.get("outdesc") or row.get("outtext")

def parse_fielding(out_text):
    if not out_text: return []
    out_text = out_text or ""
    evts = []
    m = DISMISSAL_RE["catch"].search(out_text)
    if m: evts.append((clean_name(m.group(1)), "catch"))
    m = DISMISSAL_RE["stump"].search(out_text)
    if m: evts.append((clean_name(m.group(1)), "stumping"))
    m = DISMISSAL_RE["runout"].search(out_text)
    if m:
        for n in m.group(1).split("/"):
            evts.append((clean_name(n), "runout"))
    return evts

def extract_runs_wkts_overs(inns):
    runs = try_int(inns.get("runs"))
    wkts = try_int(inns.get("wickets"))
    overs = try_float(inns.get("overs"))
    score_val = inns.get("score")
    if (runs is None or wkts is None) and score_val:
        m = SCORE_RX.search(str(score_val))
        if m:
            if runs is None: runs = int(m.group(1))
            if wkts is None: wkts = int(m.group(2) or 0)
    return runs or 0, wkts or 0, overs or 0.0

def build_team_catalog(match_info):
    # names <-> ids from match_info.team1/team2
    t1 = (match_info.get("team1") or {})
    t2 = (match_info.get("team2") or {})
    t1_id, t1_name = try_int(t1.get("teamid")), clean_name(t1.get("teamname"))
    t2_id, t2_name = try_int(t2.get("teamid")), clean_name(t2.get("teamname"))
    names_to_ids = {}
    if t1_id and t1_name: names_to_ids[t1_name.lower()] = t1_id
    if t2_id and t2_name: names_to_ids[t2_name.lower()] = t2_id
    pair = ((t1_id, t1_name), (t2_id, t2_name))
    return names_to_ids, pair

def extract_teams_from_innings(inns, match_info):
    """
    Robustly extract batting/bowling team name & id from many possible shapes.
    """
    # 1) Preferred: *teamDetails objects*
    bd = inns.get("batteamdetails") or {}
    bowld = inns.get("bowlteamdetails") or {}

    bat_name = first_non_empty(
        bd.get("batteamname"),
        inns.get("batteamname"),
        (inns.get("batteam") or {}).get("name"),
        inns.get("batteamshortname"),
    )
    bowl_name = first_non_empty(
        bowld.get("bowlteamname"),
        inns.get("bowlteamname"),
        (inns.get("bowlteam") or {}).get("name"),
        inns.get("bowlteamshortname"),
    )
    bat_id = first_non_empty(
        try_int(bd.get("batteamid")),
        try_int(inns.get("batteamid")),
        try_int((inns.get("batteam") or {}).get("id")),
    )
    bowl_id = first_non_empty(
        try_int(bowld.get("bowlteamid")),
        try_int(inns.get("bowlteamid")),
        try_int((inns.get("bowlteam") or {}).get("id")),
    )

    # 2) If we only have batting team name, deduce bowling team as "the other" from match_info
    names_to_ids, ((t1_id, t1_name), (t2_id, t2_name)) = build_team_catalog(match_info)
    if bat_name and not bowl_name:
        if t1_name and t2_name:
            if bat_name == t1_name: bowl_name = t2_name
            elif bat_name == t2_name: bowl_name = t1_name

    # 3) Fill missing IDs from name using catalog
    if bat_name and not bat_id:
        bat_id = names_to_ids.get(bat_name.lower())
    if bowl_name and not bowl_id:
        bowl_id = names_to_ids.get(bowl_name.lower())

    # Final cleanup
    bat_name = clean_name(bat_name) if bat_name else None
    bowl_name = clean_name(bowl_name) if bowl_name else None

    return bat_id, bat_name, bowl_id, bowl_name

# ---------------- Upserts ----------------
def upsert_innings(cur, match_id, innings_no, innings_id, bat_id, bat_name, bowl_id, bowl_name, runs, wkts, overs):
    cur.execute("""
        INSERT INTO match_innings (
            match_id, innings_id, innings_number,
            batting_team, bowling_team,
            batting_team_id, bowling_team_id,
            runs, wickets, overs
        ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        ON CONFLICT (match_id, innings_id) DO UPDATE SET
            innings_number = EXCLUDED.innings_number,
            batting_team = COALESCE(EXCLUDED.batting_team, match_innings.batting_team),
            bowling_team = COALESCE(EXCLUDED.bowling_team, match_innings.bowling_team),
            batting_team_id = COALESCE(EXCLUDED.batting_team_id, match_innings.batting_team_id),
            bowling_team_id = COALESCE(EXCLUDED.bowling_team_id, match_innings.bowling_team_id),
            runs = EXCLUDED.runs, wickets = EXCLUDED.wickets, overs = EXCLUDED.overs
    """, (match_id, innings_id, innings_no, bat_name, bowl_name, bat_id, bowl_id, runs, wkts, overs))

def upsert_batting(cur, match_id, innings_id, team, pos, b):
    strike = first_non_empty(b.get("strkrate"), b.get("strikerate"))
    cur.execute("""
        INSERT INTO batting_scorecard (
            match_id, innings_id, player_id, player_name, team_name,
            runs, balls_faced, fours, sixes, strike_rate,
            batting_position, dismissal, is_not_out
        ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        ON CONFLICT (match_id, innings_id, player_id) DO UPDATE SET
            team_name = COALESCE(EXCLUDED.team_name, batting_scorecard.team_name),
            runs = EXCLUDED.runs, balls_faced = EXCLUDED.balls_faced,
            fours = EXCLUDED.fours, sixes = EXCLUDED.sixes,
            strike_rate = EXCLUDED.strike_rate,
            batting_position = EXCLUDED.batting_position,
            dismissal = EXCLUDED.dismissal, is_not_out = EXCLUDED.is_not_out
    """, (
        match_id, innings_id,
        safe_player_id(b, extra=team),
        clean_name(b.get("name")),
        team,
        safe_int(b.get("runs")),
        safe_int(b.get("balls")),
        safe_int(b.get("fours")),
        safe_int(b.get("sixes")),
        safe_float(strike),
        pos,
        get_out_text(b),
        False if get_out_text(b) else True
    ))

def upsert_bowling(cur, match_id, innings_id, team, bowler):
    cur.execute("""
        INSERT INTO bowling_scorecard (
            match_id, innings_id, player_id, player_name, team_name,
            overs, maidens, runs_conceded, wickets, economy_rate
        ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        ON CONFLICT (match_id, innings_id, player_id) DO UPDATE SET
            team_name = COALESCE(EXCLUDED.team_name, bowling_scorecard.team_name),
            overs = EXCLUDED.overs, maidens = EXCLUDED.maidens,
            runs_conceded = EXCLUDED.runs_conceded, wickets = EXCLUDED.wickets,
            economy_rate = EXCLUDED.economy_rate
    """, (
        match_id, innings_id,
        safe_player_id(bowler, extra=team),
        clean_name(bowler.get("name")),
        team,
        safe_float(bowler.get("overs")),
        safe_int(bowler.get("maidens")),
        safe_int(bowler.get("runs")),
        safe_int(bowler.get("wickets")),
        safe_float(bowler.get("economy"))
    ))

def upsert_fielding(cur, match_id, innings_id, team, fielder, action):
    pid = int(hashlib.md5(f"{fielder}|{team}".encode()).hexdigest()[:8], 16)
    catches = 1 if action == "catch" else 0
    stumpings = 1 if action == "stumping" else 0
    runouts = 1 if action == "runout" else 0

    cur.execute("""
        INSERT INTO fielding_scorecard (
            match_id, innings_id, player_id, player_name, team_name,
            catches, stumpings, runouts
        ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
        ON CONFLICT (match_id, innings_id, player_id) DO UPDATE SET
            team_name = COALESCE(EXCLUDED.team_name, fielding_scorecard.team_name),
            catches = fielding_scorecard.catches + EXCLUDED.catches,
            stumpings = fielding_scorecard.stumpings + EXCLUDED.stumpings,
            runouts = fielding_scorecard.runouts + EXCLUDED.runouts
    """, (match_id, innings_id, pid, clean_name(fielder), team, catches, stumpings, runouts))

# ---------------- API ----------------
def fetch_matches(ep):
    url = f"{BASE_URL}/matches/v1/{ep}"
    try:
        r = requests.get(url, headers=HEADERS, timeout=30)
        log(f"DEBUG fetch {ep}: status={r.status_code}, length={len(r.text)}")
        if r.status_code == 204 or not r.text.strip():
            log(f"DEBUG {ep}: no content (204/empty)")
            return {}
        if not r.ok:
            log(r.text[:500])
            return {}
        return norm(r.json())
    except Exception as e:
        log(f"⚠️ Exception fetching {ep}: {e}")
        return {}

def fetch_scorecard(mid):
    url = f"{BASE_URL}/mcenter/v1/{mid}/scard"
    try:
        r = requests.get(url, headers=HEADERS, timeout=30)
        log(f"DEBUG scorecard {mid}: status={r.status_code}, length={len(r.text)}")
        if r.status_code == 204 or not r.text.strip():
            return {}
        if not r.ok:
            log(r.text[:500])
            return {}
        return norm(r.json())
    except Exception as e:
        log(f"⚠️ Exception fetching scorecard {mid}: {e}")
        return {}

# ---------------- Processing ----------------
def process_block(cur, data, label, counters):
    for tm in data.get("typematches", []):
        for sm in tm.get("seriesmatches", []):
            matches = (sm.get("seriesadwrapper") or {}).get("matches") or sm.get("matches") or []
            for m in matches:
                info = m.get("matchinfo") or {}
                mid = info.get("matchid")
                if not mid: 
                    continue
                sc = fetch_scorecard(mid)
                scards = sc.get("scorecard") or []
                if not scards:
                    continue

                for i, inns in enumerate(scards, start=1):
                    innings_id = try_int(inns.get("inningsid")) or i

                    # ✅ Extract teams robustly with fallback & deduction
                    bat_id, bat_name, bowl_id, bowl_name = extract_teams_from_innings(inns, info)

                    # Final safety: if one name missing but the other present, deduce from match_info
                    if (not bat_name or not bowl_name):
                        _, ((t1_id, t1_name), (t2_id, t2_name)) = build_team_catalog(info)
                        if not bat_name and bowl_name and t1_name and t2_name:
                            bat_name = t1_name if bowl_name == t2_name else t2_name
                        if not bowl_name and bat_name and t1_name and t2_name:
                            bowl_name = t2_name if bat_name == t1_name else t1_name
                        # fill IDs again from names if needed
                        names_to_ids, _ = build_team_catalog(info)
                        if bat_name and not bat_id:
                            bat_id = names_to_ids.get(bat_name.lower())
                        if bowl_name and not bowl_id:
                            bowl_id = names_to_ids.get(bowl_name.lower())

                    runs, wkts, overs = extract_runs_wkts_overs(inns)

                    upsert_innings(cur, mid, i, innings_id, bat_id, bat_name, bowl_id, bowl_name, runs, wkts, overs)
                    counters["innings"] += 1

                    # Batting
                    for pos, b in enumerate(inns.get("batsman") or [], start=1):
                        upsert_batting(cur, mid, innings_id, bat_name, pos, b)
                        counters["batting"] += 1
                        # Fielding attribution from dismissals -> bowling team
                        for fname, act in parse_fielding(get_out_text(b)):
                            upsert_fielding(cur, mid, innings_id, bowl_name, fname, act)
                            counters["fielding"] += 1

                    # Bowling (belongs to bowling/fielding team)
                    for bowler in inns.get("bowler") or []:
                        upsert_bowling(cur, mid, innings_id, bowl_name, bowler)
                        counters["bowling"] += 1

                counters["matches"] += 1

# ---------------- Main ----------------
def main():
    recreate_tables()
    counters = {"matches": 0, "innings": 0, "batting": 0, "bowling": 0, "fielding": 0}
    with get_conn() as conn:
        cur = conn.cursor()
        # Only recent + completed as requested
        for ep in ("recent", "completed"):
            data = fetch_matches(ep)
            if data:
                process_block(cur, data, ep, counters)
    log(f"\n✅ Insert summary: {counters}")

if __name__ == "__main__":
    main()


✅ Fresh tables created
DEBUG fetch recent: status=200, length=72815
DEBUG scorecard 130168: status=200, length=19346
DEBUG scorecard 130157: status=200, length=22941
DEBUG scorecard 130146: status=200, length=22186
DEBUG scorecard 130129: status=200, length=18468
DEBUG scorecard 130140: status=200, length=20926
DEBUG scorecard 130124: status=200, length=20600
DEBUG scorecard 134782: status=200, length=18668
DEBUG scorecard 134777: status=200, length=21562
DEBUG scorecard 134760: status=200, length=21183
DEBUG scorecard 134771: status=200, length=19618
DEBUG scorecard 105858: status=200, length=20316
DEBUG scorecard 116828: status=200, length=21309
DEBUG scorecard 131009: status=200, length=24195
DEBUG scorecard 130993: status=200, length=20416
DEBUG scorecard 130998: status=200, length=24723
DEBUG scorecard 133495: status=200, length=24240
DEBUG scorecard 133484: status=200, length=23350
DEBUG scorecard 119852: status=200, length=45522
DEBUG scorecard 113289: status=200, length=34374
D

In [ ]:
# ===========================================================
#                     Partnerships Table 
# ===========================================================


import time
import hashlib
import requests
import psycopg2

# ---------------- Config ----------------
DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "dbname": "rudra",
    "user": "postgres",
    "password": "Rudra0718",
}

BASE_URL = "https://cricbuzz-cricket.p.rapidapi.com"
HEADERS = {
    "x-rapidapi-key": "6de7b74237msh37716c5feaa0951p1eb7e3jsn0e37b5b13684",   # <-- replace with your RapidAPI key
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com",
}

# Set to True if you only want 100+ partnerships
ONLY_100_PLUS = False   # <- change to True if needed

# ---------------- Helpers ----------------
def norm(obj):
    """Lower-case all dict keys recursively so we can use a single codepath."""
    if isinstance(obj, dict):
        return { (k.lower() if isinstance(k, str) else k): norm(v) for k, v in obj.items() }
    if isinstance(obj, list):
        return [norm(x) for x in obj]
    return obj

def clean(s): 
    return (s or "").replace("†", "").strip()

def try_int(x):
    try:
        return int(x)
    except Exception:
        return None

def only_if_threshold(runs):
    """Respect the 100+ filter if enabled."""
    return (runs is not None) and (runs >= 100) if ONLY_100_PLUS else (runs is not None)

# ---------------- DB ----------------
def connect():
    return psycopg2.connect(**DB_CONFIG)

def recreate_table():
    with connect() as conn:
        cur = conn.cursor()
        cur.execute("DROP TABLE IF EXISTS partnerships;")
        cur.execute("""
            CREATE TABLE partnerships (
                id BIGSERIAL PRIMARY KEY,
                match_id BIGINT,
                match_format TEXT,
                team1_name TEXT,
                team2_name TEXT,
                innings_number INT,
                batsman1 TEXT,
                batsman2 TEXT,
                runs INT,
                balls INT,
                wicket_number INT
            );
        """)
        conn.commit()
    print("✅ Partnerships table ready")

def insert_partnership(cur, match_id, match_format, team1, team2,
                       inns_no, wicket_no, b1, b2, runs, balls):
    """Insert a partnership row (respects ONLY_100_PLUS)"""
    if not only_if_threshold(runs):
        return
    cur.execute("""
        INSERT INTO partnerships
            (match_id, match_format, team1_name, team2_name,
             innings_number, batsman1, batsman2, runs, balls, wicket_number)
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    """, (match_id, match_format, team1, team2, inns_no, b1, b2, runs, balls, wicket_no))

# ---------------- API ----------------
def safe_json(r):
    if r.status_code == 204 or not r.text.strip():
        return {}
    try:
        return r.json()
    except Exception:
        print("⚠️ JSON parse failed, snippet:", r.text[:250])
        return {}

def fetch_matches(kind):
    """kind ∈ {'recent','completed'}"""
    url = f"{BASE_URL}/matches/v1/{kind}"
    r = requests.get(url, headers=HEADERS, timeout=30)
    print(f"DEBUG fetch {kind}: {r.status_code}, length={len(r.text)}")
    return norm(safe_json(r)) if r.ok else {}

def fetch_scorecard(match_id):
    url = f"{BASE_URL}/mcenter/v1/{match_id}/scard"
    r = requests.get(url, headers=HEADERS, timeout=30)
    if r.status_code == 429:
        print(f"⚠️ 429 for match {match_id}; sleeping 8s…")
        time.sleep(8)
        return {}
    print(f"DEBUG scorecard {match_id}: {r.status_code}, length={len(r.text)}")
    return norm(safe_json(r)) if r.ok else {}

# ---------------- Core ----------------
def process_matches(cur, data):
    """
    Walks typeMatches -> seriesMatches -> matches and inserts partnerships
    (prefers API partnerships; falls back to computed from batting list).
    """
    for tblock in data.get("typematches", []):
        for s in tblock.get("seriesmatches", []):
            swrap = s.get("seriesadwrapper") or {}
            for m in (swrap.get("matches") or s.get("matches") or []):
                info = m.get("matchinfo") or {}
                match_id = info.get("matchid")
                if not match_id:
                    continue

                team1 = clean((info.get("team1") or {}).get("teamname"))
                team2 = clean((info.get("team2") or {}).get("teamname"))
                match_format = clean(info.get("matchformat"))

                sc = fetch_scorecard(match_id)
                sc_list = sc.get("scorecard") or sc.get("scorecards") or []
                if not sc_list:
                    continue

                for inns_idx, inns in enumerate(sc_list, start=1):
                    # 1) Use API partnerships if present
                    parts = inns.get("partnershipsdata") or inns.get("partnerships") or []
                    if parts:
                        print(f"🔎 match {match_id} inns {inns_idx}: partnerships from API = {len(parts)}")
                        for p in parts:
                            runs = try_int(p.get("runs")) or 0
                            balls = try_int(p.get("balls")) or 0
                            wno  = try_int(p.get("wicketno")) or 0
                            b1   = clean(p.get("batsman1name") or p.get("bat1name"))
                            b2   = clean(p.get("batsman2name") or p.get("bat2name"))
                            insert_partnership(cur, match_id, match_format, team1, team2,
                                               inns_idx, wno, b1 or "Unknown", b2 or "Unknown",
                                               runs, balls)
                        # done with this innings
                        continue

                    # 2) Fallback: compute simple pairwise partnerships from batting list
                    bats = inns.get("batsman") or inns.get("batsmendata") or []
                    if not bats:
                        print(f"⚠️ match {match_id} inns {inns_idx}: no partnerships and no batsman list")
                        continue

                    print(f"⚠️ match {match_id} inns {inns_idx}: computing partnerships from {len(bats)} batsmen")
                    # Keep original order as batting order (or use "batting_position" if present)
                    def pos(row, idx):
                        return try_int(row.get("batting_position") or row.get("position") or row.get("pos")) or (idx+1)
                    bats_sorted = sorted(list(enumerate(bats)), key=lambda t: pos(t[1], t[0]))

                    # Pair consecutive batters as a simple approximation
                    for j in range(len(bats_sorted)-1):
                        _, b1row = bats_sorted[j]
                        _, b2row = bats_sorted[j+1]
                        b1 = clean(b1row.get("name") or b1row.get("batname"))
                        b2 = clean(b2row.get("name") or b2row.get("batname"))
                        r1 = try_int(b1row.get("runs")) or 0
                        r2 = try_int(b2row.get("runs")) or 0
                        balls = (try_int(b1row.get("balls")) or 0) + (try_int(b2row.get("balls")) or 0)
                        runs = r1 + r2
                        wno  = j + 1
                        insert_partnership(cur, match_id, match_format, team1, team2,
                                           inns_idx, wno, b1 or "Unknown", b2 or "Unknown",
                                           runs, balls)

                    # be gentle with the API
                    time.sleep(1.2)

def main():
    recreate_table()
    with connect() as conn:
        cur = conn.cursor()

        # Process RECENT
        recent = fetch_matches("recent")
        process_matches(cur, recent)
        conn.commit()

        # Process COMPLETED
        completed = fetch_matches("completed")
        process_matches(cur, completed)
        conn.commit()

    print("🎉 Partnerships load complete")

if __name__ == "__main__":
    main()


✅ Partnerships table ready
DEBUG fetch recent: 200, length=72815
DEBUG scorecard 130168: 200, length=19346
⚠️ match 130168 inns 1: computing partnerships from 11 batsmen
⚠️ match 130168 inns 2: computing partnerships from 11 batsmen
DEBUG scorecard 130146: 200, length=22186
⚠️ match 130146 inns 1: computing partnerships from 11 batsmen
⚠️ match 130146 inns 2: computing partnerships from 11 batsmen
DEBUG scorecard 130140: 200, length=20926
⚠️ match 130140 inns 1: computing partnerships from 11 batsmen
⚠️ match 130140 inns 2: computing partnerships from 11 batsmen
DEBUG scorecard 130129: 200, length=18468
⚠️ match 130129 inns 1: computing partnerships from 11 batsmen
⚠️ match 130129 inns 2: computing partnerships from 11 batsmen
DEBUG scorecard 130124: 200, length=20600
⚠️ match 130124 inns 1: computing partnerships from 11 batsmen
⚠️ match 130124 inns 2: computing partnerships from 11 batsmen
DEBUG scorecard 130157: 200, length=22941
⚠️ match 130157 inns 1: computing partnerships from 1

In [ ]:
# ===========================================================
#                   Player Rankings Table 
# ===========================================================


import psycopg2
import requests
import datetime
import time

# ---------------- DB CONFIG ----------------
DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "dbname": "rudra",
    "user": "postgres",
    "password": "Rudra0718",
}

# ---------------- API CONFIG ----------------
API_KEY = "6de7b74237msh37716c5feaa0951p1eb7e3jsn0e37b5b13684"  # your RapidAPI key
HEADERS = {
    "x-rapidapi-key": API_KEY,
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}
BASE_URL = "https://cricbuzz-cricket.p.rapidapi.com/stats/v1/rankings"


# ---------------- Create Table ----------------
def init_db():
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()

    cur.execute("""
    DROP TABLE IF EXISTS player_rankings_history;

    CREATE TABLE player_rankings_history (
        player_id        BIGINT NOT NULL,
        player_name      TEXT,
        country          TEXT,
        format           TEXT NOT NULL,        -- TEST | ODI | T20I
        category         TEXT NOT NULL,        -- Batting | Bowling | All-rounder
        ranking_position INT NOT NULL,         -- 1 to 10
        rating_points    INT,
        ranking_date     DATE NOT NULL,
        created_at       TIMESTAMP WITHOUT TIME ZONE DEFAULT NOW(),
        PRIMARY KEY (format, category, ranking_position, ranking_date)
    );
    """)

    conn.commit()
    cur.close()
    conn.close()
    print("✅ Table player_rankings_history created")


# ---------------- Insert / Upsert ----------------
def save_record(fmt_db, category_db, player):
    try:
        ranking_date = datetime.datetime.strptime(player["lastUpdatedOn"], "%Y-%m-%d").date()
    except Exception:
        ranking_date = datetime.date.today()

    rating = int(player.get("rating") or 0)

    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()

    cur.execute("""
        INSERT INTO player_rankings_history
        (player_id, player_name, country, format, category, ranking_position, rating_points, ranking_date)
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
        ON CONFLICT (format, category, ranking_position, ranking_date)
        DO UPDATE SET
            player_id     = EXCLUDED.player_id,
            player_name   = EXCLUDED.player_name,
            country       = EXCLUDED.country,
            rating_points = EXCLUDED.rating_points;
    """, (
        int(player.get("id") or 0),
        player.get("name") or "Unknown",
        player.get("country") or "Unknown",
        fmt_db,
        category_db,
        int(player.get("rank") or 0),
        rating,
        ranking_date
    ))

    conn.commit()
    cur.close()
    conn.close()

    print(f"✅ {ranking_date} {fmt_db} {category_db} Rank {player['rank']}: {player['name']} ({player['country']}) → {rating}")


# ---------------- Fetch API ----------------
def fetch_rankings(fmt_api: str, category_api: str, top_n: int = 10):
    url = f"{BASE_URL}/{category_api}"
    params = {"formatType": fmt_api}

    r = requests.get(url, headers=HEADERS, params=params, timeout=20)
    if not r.ok:
        print(f"❌ API failed for {fmt_api} {category_api}: {r.status_code}")
        return []

    data = r.json() or {}
    return data.get("rank", [])[:top_n]


# ---------------- Main ----------------
def main():
    # Step 1: Create clean table
    init_db()

    # Step 2: Map API categories
    format_map = {"test": "TEST", "odi": "ODI", "t20": "T20I"}
    category_map = {"batsmen": "Batting", "bowlers": "Bowling", "allrounders": "All-rounder"}

    # Step 3: Fetch + Insert top 10 players per format/category
    for fmt_api, fmt_db in format_map.items():
        for cat_api, cat_db in category_map.items():
            players = fetch_rankings(fmt_api, cat_api, top_n=10)
            if not players:
                print(f"⚠ No data for {fmt_api} {cat_api}")
                continue

            for p in players:
                save_record(fmt_db, cat_db, p)
                time.sleep(0.1)  # small delay to be kind to API


if __name__ == "__main__":
    main()


✅ Table player_rankings_history created
✅ 2025-09-27 TEST Batting Rank 1: Joe Root (England) → 908
✅ 2025-09-27 TEST Batting Rank 2: Harry Brook (England) → 868
✅ 2025-09-27 TEST Batting Rank 3: Kane Williamson (New Zealand) → 850
✅ 2025-09-27 TEST Batting Rank 4: Steven Smith (Australia) → 816
✅ 2025-09-27 TEST Batting Rank 5: Yashasvi Jaiswal (India) → 792
✅ 2025-09-27 TEST Batting Rank 6: Temba Bavuma (South Africa) → 790
✅ 2025-09-27 TEST Batting Rank 7: Kamindu Mendis (Sri Lanka) → 781
✅ 2025-09-27 TEST Batting Rank 8: Rishabh Pant (India) → 768
✅ 2025-09-27 TEST Batting Rank 9: Daryl Mitchell (New Zealand) → 748
✅ 2025-09-27 TEST Batting Rank 10: Ben Duckett (England) → 747
✅ 2025-09-26 TEST Bowling Rank 1: Jasprit Bumrah (India) → 889
✅ 2025-09-26 TEST Bowling Rank 2: Kagiso Rabada (South Africa) → 851
✅ 2025-09-26 TEST Bowling Rank 3: Matt Henry (New Zealand) → 846
✅ 2025-09-26 TEST Bowling Rank 4: Pat Cummins (Australia) → 838
✅ 2025-09-26 TEST Bowling Rank 5: Josh Hazlewood (

In [ ]:
# ===========================================================
#                   Team Tables
# ===========================================================



import psycopg2
import requests

# ---------------- DB Config ----------------
DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "dbname": "rudra",
    "user": "postgres",
    "password": "Rudra0718",
}

# ---------------- API Config ----------------
API_KEY = "53f360dee8msh30a0da00fbde628p140c44jsn4dbf572dc823"
API_HOST = "cricbuzz-cricket.p.rapidapi.com"
HEADERS = {
    "X-RapidAPI-Key": API_KEY,
    "X-RapidAPI-Host": API_HOST
}

ENDPOINTS = [
    "https://cricbuzz-cricket.p.rapidapi.com/matches/v1/live",
    "https://cricbuzz-cricket.p.rapidapi.com/matches/v1/recent"
]

# ---------------- Create Teams Table ----------------
def create_teams_table(cur):
    cur.execute("""
    CREATE TABLE IF NOT EXISTS teams (
        team_id BIGINT PRIMARY KEY,
        team_name TEXT UNIQUE NOT NULL,
        team_sname TEXT,
        country TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """)
    print("✅ Table 'teams' created successfully")

# ---------------- Fetch Teams ----------------
def fetch_teams():
    teams_data = {}

    for url in ENDPOINTS:
        try:
            r = requests.get(url, headers=HEADERS, timeout=20)
            r.raise_for_status()
            data = r.json()

            for type_match in data.get("typeMatches", []):
                for series_item in type_match.get("seriesMatches", []):
                    series_info = series_item.get("seriesAdWrapper", {})
                    for match in series_info.get("matches", []):
                        match_info = match.get("matchInfo", {})
                        teams = [match_info.get("team1"), match_info.get("team2")]

                        for team in teams:
                            if not team:
                                continue
                            team_id = team.get("teamId")
                            team_name = team.get("teamName")
                            team_sname = team.get("teamSName")
                            country = team_name  

                            if team_id and team_name:
                                teams_data[team_id] = {
                                    "team_name": team_name,
                                    "team_sname": team_sname,
                                    "country": country
                                }
        except Exception as e:
            print(f"⚠ Error fetching teams from {url}: {e}")

    return teams_data

# ---------------- Insert Teams ----------------
def insert_teams(cur, teams_data):
    inserted, errors = 0, 0
    for tid, info in teams_data.items():
        try:
            cur.execute("""
                INSERT INTO teams (team_id, team_name, team_sname, country)
                VALUES (%s, %s, %s, %s)
                ON CONFLICT (team_id) DO UPDATE
                SET team_name = EXCLUDED.team_name,
                    team_sname = EXCLUDED.team_sname,
                    country = EXCLUDED.country
            """, (tid, info["team_name"], info["team_sname"], info["country"]))
            inserted += 1
        except Exception as e:
            print(f"⚠ Error inserting team {tid}: {e}")
            errors += 1
    print(f"✔ Inserted/Updated {inserted} teams, ⚠ {errors} errors.")

# ---------------- Main ----------------
def main():
    with psycopg2.connect(**DB_CONFIG) as conn:
        with conn.cursor() as cur:
            create_teams_table(cur)
            teams_data = fetch_teams()
            insert_teams(cur, teams_data)
            conn.commit()
    print("✅ Done, connection closed automatically.")

if __name__ == "__main__":
    main()


✅ Table 'teams' created successfully
✔ Inserted/Updated 65 teams, ⚠ 0 errors.
✅ Done, connection closed automatically.


In [ ]:
# ===========================================================
#                  Players Table 
# ===========================================================


import requests
import time
import random

# ---------------- DB Config ----------------
DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "dbname": "rudra",
    "user": "postgres",
    "password": "Rudra0718",
}

# ---------------- API Config ----------------
API_KEY = "6de7b74237msh37716c5feaa0951p1eb7e3jsn0e37b5b13684"
HEADERS = {
    "X-RapidAPI-Key": API_KEY,
    "X-RapidAPI-Host": "cricbuzz-cricket.p.rapidapi.com"
}

MATCH_ENDPOINTS = [
    "https://cricbuzz-cricket.p.rapidapi.com/matches/v1/live",
    "https://cricbuzz-cricket.p.rapidapi.com/matches/v1/recent"
]

# ---------------- Create Players Table ----------------
def create_players_table(cur):
    cur.execute("""
    CREATE TABLE IF NOT EXISTS players (
        player_id BIGINT PRIMARY KEY,
        full_name TEXT,
        nick_name TEXT,
        role TEXT,
        batting_style TEXT,
        bowling_style TEXT,
        is_keeper BOOLEAN,
        is_captain BOOLEAN,
        team_id BIGINT REFERENCES teams(team_id),
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """)
    print("✅ Table 'players' created successfully")

# ---------------- Fetch Players for a Team ----------------
def fetch_players(team_id):
    url = f"https://cricbuzz-cricket.p.rapidapi.com/teams/v1/{team_id}/players"
    try:
        r = requests.get(url, headers=HEADERS, timeout=20)
        if r.status_code != 200:
            print(f"⚠ Error {r.status_code} for team {team_id}")
            return []
        data = r.json()

        players = []
        current_role = ""
        known_roles = ["BATSMEN", "ALL ROUNDER", "WICKET KEEPER", "BOWLER"]

        for p in data.get("player", []):
            name = p.get("name")
            pid = p.get("id")

            if name in known_roles:
                current_role = name
                continue
            if not pid or not name:
                continue

            # Fallbacks
            full_name = p.get("fullName") or name
            nick_name = name
            role = current_role if current_role else "Unknown"
            batting_style = p.get("battingStyle") or "Unknown"
            bowling_style = p.get("bowlingStyle") or "Unknown"
            is_keeper = p.get("keeper", False)
            is_captain = p.get("captain", False)
            tid = p.get("teamId") or team_id

            players.append({
                "player_id": pid,
                "full_name": full_name,
                "nick_name": nick_name,
                "role": role,
                "batting_style": batting_style,
                "bowling_style": bowling_style,
                "is_keeper": is_keeper,
                "is_captain": is_captain,
                "team_id": tid
            })

        return players
    except Exception as e:
        print(f"⚠ Error fetching players for team {team_id}: {e}")
        return []

# ---------------- Insert Players ----------------
def insert_players(cur, players):
    for p in players:
        cur.execute("""
            INSERT INTO players (
                player_id, full_name, nick_name, role,
                batting_style, bowling_style, is_keeper,
                is_captain, team_id
            )
            VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
            ON CONFLICT (player_id) DO UPDATE
            SET full_name = EXCLUDED.full_name,
                nick_name = EXCLUDED.nick_name,
                role = EXCLUDED.role,
                batting_style = EXCLUDED.batting_style,
                bowling_style = EXCLUDED.bowling_style,
                is_keeper = EXCLUDED.is_keeper,
                is_captain = EXCLUDED.is_captain,
                team_id = EXCLUDED.team_id
        """, (
            p["player_id"], p["full_name"], p["nick_name"], p["role"],
            p["batting_style"], p["bowling_style"], p["is_keeper"],
            p["is_captain"], p["team_id"]
        ))

# ---------------- Main ----------------
def main():
    with psycopg2.connect(**DB_CONFIG) as conn:
        with conn.cursor() as cur:
            create_players_table(cur)

            # Step 1: Collect team IDs from matches
            team_ids = set()
            for url in MATCH_ENDPOINTS:
                r = requests.get(url, headers=HEADERS, timeout=20)
                if r.status_code != 200:
                    print(f"⚠ Error {r.status_code} for {url}")
                    continue
                data = r.json()
                for type_match in data.get("typeMatches", []):
                    for series_item in type_match.get("seriesMatches", []):
                        series_info = series_item.get("seriesAdWrapper", {})
                        for match in series_info.get("matches", []):
                            info = match.get("matchInfo", {})
                            if info.get("team1", {}).get("teamId"):
                                team_ids.add(info["team1"]["teamId"])
                            if info.get("team2", {}).get("teamId"):
                                team_ids.add(info["team2"]["teamId"])

            print(f"📌 Found {len(team_ids)} teams from matches")

            # Step 2: Fetch and insert players for each team
            total_inserted = 0
            for tid in team_ids:
                print(f"➡️ Processing team {tid}")
                players = fetch_players(tid)
                if players:
                    insert_players(cur, players)
                    conn.commit()
                    total_inserted += len(players)
                time.sleep(0.5)

            print(f"✔ Inserted/Updated {total_inserted} players")

    print("✅ Done, connection closed automatically.")

if __name__ == "__main__":
    main()


✅ Table 'players' created successfully
📌 Found 65 teams from matches
➡️ Processing team 129
⚠ Error 204 for team 129
➡️ Processing team 258
➡️ Processing team 2
➡️ Processing team 3
➡️ Processing team 5
➡️ Processing team 6
➡️ Processing team 388
➡️ Processing team 1796
⚠ Error 204 for team 1796
➡️ Processing team 9
➡️ Processing team 10
➡️ Processing team 139
⚠ Error 204 for team 139
➡️ Processing team 12
➡️ Processing team 141
⚠ Error 204 for team 141
➡️ Processing team 142
⚠ Error 204 for team 142
➡️ Processing team 14
➡️ Processing team 912
⚠ Error 204 for team 912
➡️ Processing team 145
⚠ Error 204 for team 145
➡️ Processing team 146
⚠ Error 204 for team 146
➡️ Processing team 529
⚠ Error 204 for team 529
➡️ Processing team 271
➡️ Processing team 917
⚠ Error 204 for team 917
➡️ Processing team 147
⚠ Error 204 for team 147
➡️ Processing team 151
⚠ Error 204 for team 151
➡️ Processing team 536
⚠ Error 204 for team 536
➡️ Processing team 535
⚠ Error 204 for team 535
➡️ Processing tea

In [ ]:
# ===========================================================
#                 Player_master_stats Table
# ===========================================================


import psycopg2
import requests
import re
import time
from urllib.parse import quote
from datetime import datetime, timezone

# ---------------- DB CONFIG ----------------
DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "dbname": "rudra",
    "user": "postgres",
    "password": "Rudra0718",
}

# ---------------- API CONFIG ----------------
API_KEY = "53f360dee8msh30a0da00fbde628p140c44jsn4dbf572dc823"   # 👈 replace with your RapidAPI key
HEADERS = {"x-rapidapi-key": API_KEY, "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"}
BASE_URL = "https://cricbuzz-cricket.p.rapidapi.com"

# ---------------- DB Setup ----------------
def init_db():
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    cur.execute("""
        CREATE TABLE IF NOT EXISTS player_master_stats (
            player_id BIGINT,
            format TEXT,
            player_name TEXT NOT NULL,
            team_name TEXT,
            role TEXT,
            batting_style TEXT,
            bowling_style TEXT,
            matches INT,
            innings INT,
            runs INT,
            balls_faced INT,
            hundreds INT,
            fifties INT,
            highest_score INT,
            batting_average DECIMAL(6,2),
            strike_rate DECIMAL(6,2),
            not_outs INT,
            ducks INT,
            wickets INT,
            balls_bowled INT,
            runs_conceded INT,
            bowling_average DECIMAL(6,2),
            economy_rate DECIMAL(6,2),
            four_wicket_hauls INT,
            five_wicket_hauls INT,
            ten_wicket_hauls INT,
            best_bowling_innings TEXT,
            best_bowling_match TEXT,
            catches INT,
            stumpings INT,
            icc_bat_best_rank INT,
            icc_bowl_best_rank INT,
            icc_allround_best_rank INT,
            created_at TIMESTAMP,
            PRIMARY KEY (player_id, format)
        )
    """)
    conn.commit()
    cur.close()
    conn.close()
    print("✅ Table player_master_stats ready")

def save_to_db(record):
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    sql = """
        INSERT INTO player_master_stats (
            player_id, format, player_name, team_name, role,
            batting_style, bowling_style,
            matches, innings, runs, balls_faced,
            hundreds, fifties, highest_score, batting_average, strike_rate,
            not_outs, ducks,
            wickets, balls_bowled, runs_conceded, bowling_average, economy_rate,
            four_wicket_hauls, five_wicket_hauls, ten_wicket_hauls,
            best_bowling_innings, best_bowling_match,
            catches, stumpings,
            icc_bat_best_rank, icc_bowl_best_rank, icc_allround_best_rank,
            created_at
        )
        VALUES (%s,%s,%s,%s,%s,%s,%s,
                %s,%s,%s,%s,
                %s,%s,%s,%s,%s,
                %s,%s,
                %s,%s,%s,%s,%s,
                %s,%s,%s,
                %s,%s,
                %s,%s,
                %s,%s,%s,
                %s)
        ON CONFLICT (player_id, format) DO UPDATE SET
            matches=EXCLUDED.matches,
            innings=EXCLUDED.innings,
            runs=EXCLUDED.runs,
            balls_faced=EXCLUDED.balls_faced,
            hundreds=EXCLUDED.hundreds,
            fifties=EXCLUDED.fifties,
            highest_score=EXCLUDED.highest_score,
            batting_average=EXCLUDED.batting_average,
            strike_rate=EXCLUDED.strike_rate,
            not_outs=EXCLUDED.not_outs,
            ducks=EXCLUDED.ducks,
            wickets=EXCLUDED.wickets,
            balls_bowled=EXCLUDED.balls_bowled,
            runs_conceded=EXCLUDED.runs_conceded,
            bowling_average=EXCLUDED.bowling_average,
            economy_rate=EXCLUDED.economy_rate,
            four_wicket_hauls=EXCLUDED.four_wicket_hauls,
            five_wicket_hauls=EXCLUDED.five_wicket_hauls,
            ten_wicket_hauls=EXCLUDED.ten_wicket_hauls,
            best_bowling_innings=EXCLUDED.best_bowling_innings,
            best_bowling_match=EXCLUDED.best_bowling_match,
            catches=EXCLUDED.catches,
            stumpings=EXCLUDED.stumpings,
            icc_bat_best_rank=EXCLUDED.icc_bat_best_rank,
            icc_bowl_best_rank=EXCLUDED.icc_bowl_best_rank,
            icc_allround_best_rank=EXCLUDED.icc_allround_best_rank,
            created_at=EXCLUDED.created_at;
    """
    cur.execute(sql, record)
    conn.commit()
    cur.close()
    conn.close()

# ---------------- API Helpers ----------------
def search_player(query):
    url = f"{BASE_URL}/stats/v1/player/search?plrN={quote(query)}"
    r = requests.get(url, headers=HEADERS)
    return r.json().get("player", [])[0] if r.status_code == 200 and r.json().get("player") else None

def get_player_profile(pid):
    url = f"{BASE_URL}/stats/v1/player/{pid}"
    r = requests.get(url, headers=HEADERS)
    return r.json() if r.status_code == 200 else {}

def get_stats(pid, stat_type):
    url = f"{BASE_URL}/stats/v1/player/{pid}/{stat_type}"
    r = requests.get(url, headers=HEADERS)
    return r.json() if r.status_code == 200 else {}

def extract_stats_table(stats_json):
    if not stats_json or "headers" not in stats_json or "values" not in stats_json:
        return {}
    headers = stats_json["headers"][1:]
    stats = {}
    for row in stats_json["values"]:
        row_vals = row["values"]
        rowheader = row_vals[0].strip()
        for i, fmt in enumerate(headers, start=1):
            stats.setdefault(fmt, {})[rowheader] = row_vals[i]
    return stats

# ---------------- Normalizers ----------------
def to_int(v):
    try:
        return int(re.sub(r"[^0-9]", "", str(v))) if v else 0
    except:
        return 0

def to_float(v):
    try:
        return float(str(v).replace("-", "0")) if v else 0.0
    except:
        return 0.0

# ---------------- Mapping ----------------
def map_stats(bat, bowl, fld, fmt):
    """Map raw Cricbuzz stats → DB columns"""
    b = bat.get(fmt, {})
    bw = bowl.get(fmt, {})
    f = fld.get(fmt, {})

    return {
        "matches": to_int(b.get("Matches")),
        "innings": to_int(b.get("Innings")),
        "runs": to_int(b.get("Runs")),
        "balls_faced": to_int(b.get("Balls") or b.get("BF")),
        "hundreds": to_int(b.get("100s")),
        "fifties": to_int(b.get("50s")),
        "highest_score": to_int(b.get("Highest") or 0),
        "batting_average": to_float(b.get("Average")),
        "strike_rate": to_float(b.get("SR")),
        "not_outs": to_int(b.get("Not Out") or b.get("NO")),
        "ducks": to_int(b.get("Ducks") or b.get("0")),
        "wickets": to_int(bw.get("Wickets") or bw.get("Wkts")),
        "balls_bowled": to_int(bw.get("Balls")),
        "runs_conceded": to_int(bw.get("Runs")),
        "bowling_average": to_float(bw.get("Average") or bw.get("Avg")),
        "economy_rate": to_float(bw.get("Eco") or bw.get("Econ")),
        "four_wicket_hauls": to_int(bw.get("4w")),
        "five_wicket_hauls": to_int(bw.get("5w")),
        "ten_wicket_hauls": to_int(bw.get("10w")),
        "best_bowling_innings": bw.get("BBI",""),
        "best_bowling_match": bw.get("BBM",""),
        "catches": to_int(f.get("Ct") or f.get("Catches")),
        "stumpings": to_int(f.get("St") or f.get("Stumpings"))
    }

# ---------------- Main ----------------
def main():
    init_db()

    players_list = ["Sachin Tendulkar","Jacques Kallis","Rahul Dravid","Brian Lara","Ricky Ponting",
        "Virat Kohli","Kumar Sangakkara","Joe Root","Steven Smith","Kane Williamson",
        "AB de Villiers","Mahela Jayawardene","Chris Gayle","Rohit Sharma","Jos Buttler",
        "Suryakumar Yadav","Yashasvi Jaiswal","Travis Head","David Warner","Babar Azam",
        "Adam Gilchrist","Muttiah Muralitharan","Shane Warne","Wasim Akram","Glenn McGrath",
        "MS Dhoni","Allan Border","Inzamam-ul-Haq","Saeed Anwar","Anil Kumble",
        "Rashid Khan","Jacques Rudolph","Michael Clarke","Kevin Pietersen","Javed Miandad",
        "Ben Stokes","Shahid Afridi","Lasith Malinga","Dwayne Bravo","Imran Khan"]

    for name in players_list:
        player = search_player(name)
        if not player:
            print(f"❌ Not found: {name}")
            continue

        pid = int(player["id"])
        pname = player["name"]
        team = player.get("teamName","Unknown")

        profile = get_player_profile(pid)
        role = profile.get("role","Unknown")
        bat_style = profile.get("bat","Unknown")
        bowl_style = profile.get("bowl","Unknown")

        rankings = profile.get("rankings", {})
        icc_bat_best = rankings.get("bat", {}).get("testBestRank") or rankings.get("bat", {}).get("odiBestRank")
        icc_bowl_best = rankings.get("bowl", {}).get("testBestRank") or rankings.get("bowl", {}).get("odiBestRank")
        icc_all_best = rankings.get("all", {}).get("testBestRank") or rankings.get("all", {}).get("odiBestRank")

        bat = extract_stats_table(get_stats(pid, "batting"))
        bowl = extract_stats_table(get_stats(pid, "bowling"))
        fld = extract_stats_table(get_stats(pid, "fielding"))

        # Merge T20 → T20I
        for ds in (bat,bowl,fld):
            if "T20" in ds:
                ds.setdefault("T20I", {}).update(ds.pop("T20"))

        for fmt in ["Test","ODI","T20I","IPL"]:
            if fmt in bat or fmt in bowl or fmt in fld:
                stats = map_stats(bat,bowl,fld,fmt)

                record = [
                    pid, fmt, pname, team, role, bat_style, bowl_style,
                    stats["matches"], stats["innings"], stats["runs"], stats["balls_faced"],
                    stats["hundreds"], stats["fifties"], stats["highest_score"], stats["batting_average"], stats["strike_rate"],
                    stats["not_outs"], stats["ducks"],
                    stats["wickets"], stats["balls_bowled"], stats["runs_conceded"], stats["bowling_average"], stats["economy_rate"],
                    stats["four_wicket_hauls"], stats["five_wicket_hauls"], stats["ten_wicket_hauls"],
                    stats["best_bowling_innings"], stats["best_bowling_match"],
                    stats["catches"], stats["stumpings"],
                    icc_bat_best, icc_bowl_best, icc_all_best,
                    datetime.now(timezone.utc)
                ]

                save_to_db(record)
                print(f"✅ Saved {pname} {fmt}")

        time.sleep(1.5)

if __name__ == "__main__":
    main()


✅ Table player_master_stats ready
✅ Saved Sachin Tendulkar Test
✅ Saved Sachin Tendulkar ODI
✅ Saved Sachin Tendulkar T20I
✅ Saved Sachin Tendulkar IPL
✅ Saved Jacques Kallis Test
✅ Saved Jacques Kallis ODI
✅ Saved Jacques Kallis T20I
✅ Saved Jacques Kallis IPL
✅ Saved Rahul Dravid Test
✅ Saved Rahul Dravid ODI
✅ Saved Rahul Dravid T20I
✅ Saved Rahul Dravid IPL
✅ Saved Brian Lara Test
✅ Saved Brian Lara ODI
✅ Saved Brian Lara T20I
✅ Saved Brian Lara IPL
✅ Saved Ricky Ponting Test
✅ Saved Ricky Ponting ODI
✅ Saved Ricky Ponting T20I
✅ Saved Ricky Ponting IPL
✅ Saved Virat Kohli Test
✅ Saved Virat Kohli ODI
✅ Saved Virat Kohli T20I
✅ Saved Virat Kohli IPL
✅ Saved Kumar Sangakkara Test
✅ Saved Kumar Sangakkara ODI
✅ Saved Kumar Sangakkara T20I
✅ Saved Kumar Sangakkara IPL
✅ Saved Joe Root Test
✅ Saved Joe Root ODI
✅ Saved Joe Root T20I
✅ Saved Joe Root IPL
✅ Saved Steven Smith Test
✅ Saved Steven Smith ODI
✅ Saved Steven Smith T20I
✅ Saved Steven Smith IPL
✅ Saved Kane Williamson Test
✅ S